In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn,rnn_cell
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.001
epochs = 100000
batch_size = 128
display_step = 10

n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

X = tf.placeholder(tf.float32, [None, n_steps, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])

In [3]:
weights = {
    'out' : tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out' : tf.Variable(tf.random_normal([n_classes])) 
}

In [4]:
def RNN(X, weights, biases):
    X = tf.transpose(X, [1,0,2])
    X = tf.reshape(X, [-1, n_input])
    X = tf.split(0, n_steps, X)
    
    with tf.variable_scope('forward'):
        lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias = 1.0)
        outputs, states = rnn.rnn(lstm_cell, X, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(X, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Y, 1))
accuracy= tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.initialize_all_variables()


In [5]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    while step*batch_size < epochs:
        batch_X, batch_Y = mnist.train.next_batch(batch_size)
        batch_X = batch_X.reshape((batch_size, n_steps, n_input))
        
        sess.run(optimizer, feed_dict={X:batch_X, Y: batch_Y})
        
        if step% display_step == 0:
            acc = sess.run(accuracy, feed_dict={X:batch_X, Y:batch_Y})
            loss = sess.run(cost, feed_dict={X:batch_X, Y:batch_Y})
            
            print( "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step +=1
        
    print("Optimization Finished!")
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy: "+ str(sess.run(accuracy, feed_dict={X:test_data,Y:test_label})))

Iter 1280, Minibatch Loss= 1.607635, Training Accuracy= 0.41406
Iter 2560, Minibatch Loss= 1.467131, Training Accuracy= 0.48438
Iter 3840, Minibatch Loss= 1.097093, Training Accuracy= 0.60156
Iter 5120, Minibatch Loss= 0.817938, Training Accuracy= 0.78125
Iter 6400, Minibatch Loss= 0.703708, Training Accuracy= 0.82031
Iter 7680, Minibatch Loss= 1.095960, Training Accuracy= 0.63281
Iter 8960, Minibatch Loss= 0.665088, Training Accuracy= 0.78125
Iter 10240, Minibatch Loss= 0.569227, Training Accuracy= 0.80469
Iter 11520, Minibatch Loss= 0.333717, Training Accuracy= 0.92188
Iter 12800, Minibatch Loss= 0.594469, Training Accuracy= 0.81250
Iter 14080, Minibatch Loss= 0.365413, Training Accuracy= 0.85938
Iter 15360, Minibatch Loss= 0.333534, Training Accuracy= 0.89062
Iter 16640, Minibatch Loss= 0.358673, Training Accuracy= 0.92969
Iter 17920, Minibatch Loss= 0.314237, Training Accuracy= 0.89062
Iter 19200, Minibatch Loss= 0.263935, Training Accuracy= 0.89844
Iter 20480, Minibatch Loss= 0.16

In [29]:
from pprint import pprint
sess = tf.InteractiveSession()
X = np.array([
                [[1, 2, 3,4],
                 [5,  6, 7,8],
                 [9,10,11,12]],
        
                [[13,14,15,16],
                 [17, 18, 19,20],
                 [21, 22, 23, 24]]
             ])

pprint(X) # 2x 3x 4

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]])


In [30]:
pprint(tf.transpose(X,[0,2,1]).eval())

array([[[ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11],
        [ 4,  8, 12]],

       [[13, 17, 21],
        [14, 18, 22],
        [15, 19, 23],
        [16, 20, 24]]])


In [31]:
pprint(tf.transpose(X,[2,1,0]).eval())#3x2x4

array([[[ 1, 13],
        [ 5, 17],
        [ 9, 21]],

       [[ 2, 14],
        [ 6, 18],
        [10, 22]],

       [[ 3, 15],
        [ 7, 19],
        [11, 23]],

       [[ 4, 16],
        [ 8, 20],
        [12, 24]]])


In [32]:
pprint(tf.transpose(X,[2,1,0]).eval()) # 4x3x2

array([[[ 1, 13],
        [ 5, 17],
        [ 9, 21]],

       [[ 2, 14],
        [ 6, 18],
        [10, 22]],

       [[ 3, 15],
        [ 7, 19],
        [11, 23]],

       [[ 4, 16],
        [ 8, 20],
        [12, 24]]])


In [33]:
pprint(tf.transpose(X,[1,0,2]).eval()) # 3x2x4

array([[[ 1,  2,  3,  4],
        [13, 14, 15, 16]],

       [[ 5,  6,  7,  8],
        [17, 18, 19, 20]],

       [[ 9, 10, 11, 12],
        [21, 22, 23, 24]]])


In [35]:
A=tf.transpose(X,[1,0,2]).eval()
tf.reshape(A, [-1, 4]).eval()

array([[ 1,  2,  3,  4],
       [13, 14, 15, 16],
       [ 5,  6,  7,  8],
       [17, 18, 19, 20],
       [ 9, 10, 11, 12],
       [21, 22, 23, 24]])